In [ ]:
! pip install opencv-python mediapipe tensorflow keras numpy stockfish python-chess --break-system-packages


In [ ]:
! pip install requests


In [ ]:
import cv2
import numpy as np
from transformers import AutoImageProcessor, AutoModelForObjectDetection
import torch
from torch import nn
import requests
import json

# Lichess API URL for Stockfish analysis

# Load pre-trained model and processor
processor = AutoImageProcessor.from_pretrained("aesat/detr-finetuned-chess")
model = AutoModelForObjectDetection.from_pretrained("aesat/detr-finetuned-chess")


def process_chessboard_image(image_path):
    """Process the chessboard image and detect pieces."""
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to RGB (required by the processor)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Preprocess the image using the processor
    inputs = processor(images=image_rgb, return_tensors="pt")

    # Perform inference using the pre-trained model
    with torch.no_grad():
        outputs = model(**inputs)

    # Process the detections to identify piece positions
    piece_positions = detect_pieces_from_detections(outputs, image)

    # Convert piece positions to FEN
    fen = generate_fen_from_positions(piece_positions)
    return fen


def detect_pieces_from_detections(outputs, image):
    """Extract piece positions from the detection results."""
    piece_positions = {}

    # Extract the bounding boxes and logits (for classification scores)
    boxes = outputs.pred_boxes[0].cpu().numpy()  # bounding boxes
    logits = outputs.logits[0].cpu().numpy()  # classification logits

    # Apply softmax to get class probabilities
    softmax = nn.Softmax(dim=-1)
    probs = softmax(torch.tensor(logits)).cpu().numpy()

    # Define the confidence threshold
    confidence_threshold = 0.5

    # Map the labels to chess pieces (you may need to refer to the model documentation for exact labels)
    piece_mapping = {
        1: "P",  # Pawn
        2: "R",  # Rook
        3: "N",  # Knight
        4: "B",  # Bishop
        5: "Q",  # Queen
        6: "K",  # King
    }

    # Convert bounding boxes to chessboard positions (you may need additional logic here)
    height, width, _ = image.shape
    for i, prob in enumerate(probs):
        if np.max(prob) > confidence_threshold:  # Only consider high-confidence detections
            label = np.argmax(prob)  # Get the predicted label (class)
            piece = piece_mapping.get(label, "")
            if piece:
                # Convert box coordinates to board coordinates
                x_min, y_min, x_max, y_max = boxes[i]
                square = convert_box_to_square(x_min, y_min, x_max, y_max, width, height)
                piece_positions[square] = piece

    return piece_positions


def convert_box_to_square(x_min, y_min, x_max, y_max, width, height):
    """Convert bounding box coordinates to chessboard square."""
    # Calculate the square based on the bounding box position
    col = int((x_min + x_max) / 2 / width * 8)
    row = int((y_min + y_max) / 2 / height * 8)
    square = f"{chr(97 + col)}{8 - row}"  # Convert to chessboard notation (e.g., e2)
    return square


def generate_fen_from_positions(positions):
    """Generate a FEN string from detected piece positions."""
    board = [["" for _ in range(8)] for _ in range(8)]

    # Map the positions to the chessboard grid
    for square, piece in positions.items():
        row = 8 - int(square[1])  # FEN rows start from the bottom
        col = ord(square[0]) - ord('a')  # Convert 'a'-'h' to 0-7
        board[row][col] = piece

    # Create the FEN string
    fen_rows = []
    for row in board:
        empty = 0
        fen_row = ""
        for cell in row:
            if cell == "":
                empty += 1
            else:
                if empty > 0:
                    fen_row += str(empty)
                    empty = 0
                fen_row += cell
        if empty > 0:
            fen_row += str(empty)
        fen_rows.append(fen_row)

    # Add the default FEN suffix (White to move, no castling rights, no en passant, 0 half-moves, 1 full-move)
    return "/".join(fen_rows) + " w - - 0 1"


def get_chess_api_move(fen, variants=1, depth=12, max_thinking_time=50):
    # Define the API endpoint
    url = "https://chess-api.com/v1"
    
    # Define the payload with the FEN and optional parameters
    payload = {
        "fen": fen,
        "variants": variants,
        "depth": depth,
        "maxThinkingTime": max_thinking_time,
    }
    
    try:
        # Make a POST request to the API
        response = requests.post(url, json=payload, headers={"Content-Type": "application/json"})
        
        # Check if the response is successful
        if response.status_code == 200:
            # Parse and return the response data
            data = response.json()
            return data
        else:
            # Print error details
            print(f"Error: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"An exception occurred: {e}")
        return None

# Load and process the chessboard image
image_path = "chessboard.jpg"  # Replace with your chessboard image path
fen = process_chessboard_image(image_path)

if fen:
    print(f"Generated FEN: {fen}")
    fen = "8/1P1R4/n1r2B2/3Pp3/1k4P1/6K1/Bppr1P2/2q5 w - - 0 1"

    # Get the best move from Stockfish API
    result=get_chess_api_move(fen)
    if result:
        print("Response from Chess API:")
        # print(json.dumps(result, indent=4))  # Pretty print the JSON response
    
    # Extract and display relevant move details
        best_move = result.get("move", "No move found")
        eval_score = result.get("eval", "No evaluation score")
        continuation = result.get("continuationArr", [])
        print(f"Best Move: {best_move}")
        print(f"Evaluation Score: {eval_score}")
        print(f"Continuation: {continuation}")
    else:
        print("Failed to retrieve a move.")
else:
    print("Failed to process the chessboard image.")


Generated FEN: Q7/8/8/8/8/8/8/8 w - - 0 1
Response from Chess API:
Best Move: b7b8q
Evaluation Score: -10.43
Continuation: ['a6b8', 'd7b7', 'b4a3', 'd5c6', 'b8c6', 'g3h4', 'a3a2', 'h4h5', 'c1h1', 'f6h4', 'c2c1q', 'b7b6', 'h1d5']
